In [24]:
## Import Libraries, and read the key-token
import openai
import os
from dotenv import load_dotenv
import tempfile
import requests
import re

## Load the Key token
_ = load_dotenv()
key_token = os.getenv('OpenAI_KEY_TOKEN')

## Assign that key_token to api_key of OpenAI
openai.api_key = key_token

In [11]:
## paramas
ingredients = ''.join(input('Enter your Ingredients Here with space between them. !'))
kcal = 2000

## prompts
system_prompt = 'You are a smart and a telented cook'
user_prompt = f'''Create a healthy daily meal plan for breakfast, lunch and dinner based on
                    the following ingredients {ingredients} with only 3 meals.
                    Explain each recipe.
                    The total daily intake of kcal should be below {kcal}.
                    Assign a suggestive and concise title to each meal.
                    Your answer should end with 'Titles: ' and the title of each recipe.
                    Return the 3 Titles at the End of the response. '''
## messages
messages = [ 
    {'role': 'system', 'content': system_prompt}, 
    {'role': 'user', 'content': user_prompt}
        ]

## Using ChatGPT for ChatCompletion
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',           
            messages=messages,
            temperature=0.7,  
            max_tokens=1000,  
                )

response = response['choices'][0]['message']['content']  

print(response)

Here is a healthy daily meal plan based on the given ingredients:

Breakfast: Baked Eggs with Spinach and Tomatoes (400 kcal)
- Preheat the oven to 375°F (190°C).
- In a skillet, heat 1 tablespoon of oil over medium heat.
- Add 1 cup of spinach and 1 diced tomato. Cook until spinach is wilted and tomato is softened.
- Transfer the spinach and tomato to a small baking dish.
- Crack 2 eggs into the dish and bake for 12-15 minutes or until the eggs are set.
- Serve with a slice of whole-grain bread.

Lunch: Grilled Chicken Salad (500 kcal)
- Preheat a grill or grill pan to medium-high heat.
- Season 4 ounces of chicken breast with salt and pepper.
- Grill the chicken for 6-8 minutes per side or until cooked through.
- In a large bowl, mix 2 cups of mixed greens, 1/2 cup of cherry tomatoes, and 1/4 cup of sliced cucumbers.
- Whisk together 1 tablespoon of oil and 1 tablespoon of vinegar to make a dressing.
- Slice the grilled chicken and add it to the salad. Drizzle with the dressing.

Din

In [25]:
## Generate Image for these meals using Dall-E
meals_titles = response.splitlines()[-3:]

## Remove Numbers if exists
meals_titles = [re.sub(r'\d+.', '', title).strip() for title in meals_titles]
meals_titles

['Baked Eggs with Spinach and Tomatoes',
 'Grilled Chicken Salad',
 'Baked Fish with Roasted Vegetables']

In [26]:
## Create the folder for saving images locally
os.makedirs('gen_images', exist_ok=True)

In [27]:
## Usign Dall-E -> for Image 1 (BreakFast)
response = openai.Image.create(
                prompt=meals_titles[0],   ## The BreakFast the fisrt image
                n=1,    
                size='1024x1024'    
            )

imag_url = response['data'][0]['url']  ## Getting URL only

## Download to disk with temporary filename
request_res = requests.get(imag_url)

## Check response and save the image to (gen_images) folder
if request_res.status_code == 200:

    img_temp_name = os.path.basename(tempfile.NamedTemporaryFile(suffix=".png").name)
    ## Set the path where you want to save the image
    image_path_local = os.path.join('gen_images', img_temp_name)
    
    ## Write the image to the save path
    with open(image_path_local, "wb") as file:
        file.write(request_res.content)
        
else:
    print('There is an error in getting the image link.')